In [1]:
pip install ibmdbpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 148 kB 4.7 MB/s 
  Created wheel for pypyodbc: filename=pypyodbc-1.3.6-py3-none-any.whl size=22873 sha256=9cae984aaf5618d1baa65691e7025b17eaff7dffd5b90cfdc962fa172c6dad29
  Stored in directory: /root/.cache/pip/wheels/83/ef/f2/c2cc4edd495948cddf26ce772b1ffcbda9c649358b0de9dad5
Successfully built pypyodbc


In [2]:
pip install arff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for arff: filename=arff-0.9-py3-none-any.whl size=4971 sha256=abde406d8b8f0ef6c2916bebbcae6fff8aaa7a37e680ea02deb264cd6ac09d51
  Stored in directory: /root/.cache/pip/wheels/fe/81/bd/4ae90e24ba860304e375da219f9205b2586dbee255f3ee70e2
Successfully built arff


In [3]:
pip install -U ITMO_FS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70 kB 3.3 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 


In [4]:
import ibmdbpy
import pandas as pd
import numpy as np
from ibmdbpy.feature_selection import su
from scipy.io import arff
from ITMO_FS.filters.univariate import su_measure

/usr/local/lib/python3.7/dist-packages/ibmdbpy/frame.py:45: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index, RangeIndex


In [5]:
raw = arff.loadarff('corral.arff')
df = pd.DataFrame(raw[0])
df = df.astype(int)

In [6]:
df

,A0,A1,B0,B1,Irrelevant,Correlated,class
0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,1,0,0
4,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...
155,1,1,1,0,1,1,1
156,1,1,1,1,0,0,1
157,1,1,1,1,0,1,1
158,1,1,1,1,1,0,1


In [7]:
su_basic = su_measure(np.array(df[["A1", "B1", "A0", "B0", "Irrelevant", "Correlated"]]), np.array(df["class"]))
su_basic.size

6

In [8]:
s_list = np.array([])

In [9]:
s_list = np.append(s_list, 3)

In [78]:
def FCBF_algo(df, sigma):
  n = df.shape[1]
  su_basic = su_measure(np.array(df[["A0", "A1", "B0", "B1", "Irrelevant", "Correlated"]]), np.array(df["class"]))
  
  s_list = []

  for su, name in zip(su_basic, np.array(df.columns)[:np.array(df.columns).size-1]):
    if (su > sigma):
      s_list.append([su, name])
  
  s_list.sort(reverse=True)
  print("Set at the start:", s_list)
  s_list.append([np.NaN, np.NaN])
  
  j = 0
  F_j = s_list[j][1]
  while True:
    i = j + 1
    F_i = s_list[i][1]
    if (not(pd.isna(F_i))):
      while True:
        print("Compairing F_", i, " : ", s_list[i][1], " to F_", j, " : ", s_list[j][1], sep = '') 
        if (su_measure(np.array(df[[F_i]]), np.array(df[F_j])) >= s_list[i][0]):
          print("Удалён элемент: ", s_list[i][1])
          s_list.pop(i)
          i -= 1
        i += 1
        F_i = s_list[i][1]
        if (pd.isna(F_i)):
          break
    j += 1
    F_j = s_list[j][1]
    if (pd.isna(F_j)):
      break
  return s_list

In [79]:
fin_list = FCBF_algo(df, 0)
print(fin_list)

Set at the start: [[0.5912979324490651, 'Correlated'], [0.5503811930164574, 'B1'], [0.5503811930164574, 'B0'], [0.5503811930164574, 'A1'], [0.5503811930164574, 'A0'], [0.4971587984427401, 'Irrelevant']]
Compairing F_1 : B1 to F_0 : Correlated
Compairing F_2 : B0 to F_0 : Correlated
Compairing F_3 : A1 to F_0 : Correlated
Compairing F_4 : A0 to F_0 : Correlated
Compairing F_5 : Irrelevant to F_0 : Correlated
Удалён элемент:  Irrelevant
Compairing F_2 : B0 to F_1 : B1
Compairing F_3 : A1 to F_1 : B1
Compairing F_4 : A0 to F_1 : B1
Compairing F_3 : A1 to F_2 : B0
Compairing F_4 : A0 to F_2 : B0
Compairing F_4 : A0 to F_3 : A1
[[0.5912979324490651, 'Correlated'], [0.5503811930164574, 'B1'], [0.5503811930164574, 'B0'], [0.5503811930164574, 'A1'], [0.5503811930164574, 'A0'], [nan, nan]]
